<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/learningSpark6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!pip install py4j

     |████████████████████████████████| 204kB 9.8MB/s 


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
comments1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBcomments.csv", inferSchema=True, header=True)

In [0]:
comments2= spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/UScomments.csv", inferSchema=True, header=True)

In [11]:
comments1.show()

+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|jt2OHQh0HoQ|It's more accurat...|    0|      0|
|jt2OHQh0HoQ|To be there with ...|    1|      0|
|jt2OHQh0HoQ|Thank gosh, a pla...|    0|      0|
|jt2OHQh0HoQ|What happened to ...|    0|      0|
|jt2OHQh0HoQ|Power is the dise...|    0|      0|
|jt2OHQh0HoQ|Keep calm and buy...|    0|      0|
|jt2OHQh0HoQ|i am a big fan of...|    0|      0|
|jt2OHQh0HoQ|You will never fi...|    0|      0|
|jt2OHQh0HoQ|*APPLE JUST COMME...|    0|      0|
|jt2OHQh0HoQ|I'm only here to ...|    0|      0|
|jt2OHQh0HoQ|Mom said: Just on...|    0|      0|
|jt2OHQh0HoQ|what is the song?...|    0|      0|
|jt2OHQh0HoQ|I started crying ...|    0|      0|
|jt2OHQh0HoQ|3 years late on w...|    0|      0|
|jt2OHQh0HoQ|more one before t...|    0|      0|
|jt2OHQh0HoQ|subscribe me if y...|    0|      0|
|jt2OHQh0HoQ|They really just ...|    0|      0|
|jt2OHQh0HoQ|so if s

In [12]:
type(comments1)

pyspark.sql.dataframe.DataFrame

In [0]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

comments=unionAll(comments1,comments2)

In [14]:
comments.count()

1410466

In [15]:
comments_rdd = comments.select('comment_text').rdd.flatMap(lambda x: x)
comments_rdd.collect()

[u"It's more accurate to call it the M+ (1000) because the price is closer than calling it the X (10).",
 u'To be there with a samsung phone\\n\U0001f602\U0001f602\U0001f602',
 u'Thank gosh, a place I can watch it without having to be at HD... my speed doesn\u2019t support HD',
 u'What happened to the home button on the iPhone X? *****Cough****copying Samsung******cough',
 u'Power is the disease.\xa0 Care is the cure.\xa0 Keep caring for yourself and others as best as you can.\xa0 This is life.',
 u'Keep calm and buy iphone 8 Keep calm and buy iphone 8 plus Keep calm and buy iphone X. What is your favourite?',
 u'i am a big fan of youtube and u !!!!!!!!!!!!!',
 u'You will never find Losers who line up and pay good money for this crap line up to do charity or community work.',
 u"*APPLE JUST COMMENTED ON MY LAST VIDEO* I'm crying right now \U0001f62d\U0001f62d\U0001f62d\U0001f622\U0001f622\U0001f622",
 u"I'm only here to see Emma, I love her so much! i'm so proud of her! :)",
 u'Mom sai

In [0]:
header = comments_rdd.first()

In [0]:
data_rmv_col = comments_rdd.filter(lambda row: row != header)

In [0]:
lowerCase_sentRDD = data_rmv_col.map(lambda x : x.lower())

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)
    
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [0]:
sentenceTokenizeRDD.count()